# Data catalog export

https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_get_table

Generated with Gemini and tested

Luis Gerardo Baeza <br>
Jun 18, 2024 <br>
Use on your own responsibility

In [ ]:
pip install google-cloud-datacatalog

In [ ]:
from google.cloud import bigquery
from google.cloud import datacatalog_v1

In [ ]:
project_id = ""
dataset_id = ""

bq_client = bigquery.Client(project=project_id)
datacatalog_client = datacatalog_v1.PolicyTagManagerClient()

dataset_ref = bq_client.dataset(dataset_id)
tables = bq_client.list_tables(dataset_ref)
tables

### Print table schema information

In [ ]:
table_info = []
for table in tables:
    table_ref = dataset_ref.table(table.table_id)
    table_obj = bq_client.get_table(table_ref)  # Get table details
    
    # Extract Schema
    schema_info = []
    for field in table_obj.schema:
        policy_tags = field.policy_tags.names if field.policy_tags else None
        schema_info.append({
            'name': field.name,
            'type': field.field_type,
            'mode': field.mode,
            'policy_tags': policy_tags
        })
        
        if policy_tags != None: 
            print(policy_tags)

    # Store Table Information
    table_info.append({
        'table_id': table.table_id,
        'schema': schema_info,
    })

### Find the related policy tag information

In [ ]:
# Replace with the taxonomy id got from previous line in the form of 
# projects/project_id/locations/location/taxonomies/taxonomy_id
taxonomy_path = ''

In [ ]:
# Replace with the policy tag id got from previous line in the form of 
# projects/project_id/locations/location/taxonomies/taxonomy_id/policyTags/tag_id
policy_tag_path = ''

In [ ]:
# Get Taxonomy info
datacatalog_client.get_taxonomy(name=taxonomy_path)

In [ ]:
# Get the policy tag name
datacatalog_client.get_policy_tag(name=policy_tag_path)